In [894]:
!git add "Introduction\ to\ Pandas.ipynb"

In [895]:
!git commit -m "updating Pandas introduction"

[master 1eac305] updating Pandas introduction
 1 file changed, 2352 insertions(+), 57 deletions(-)


In [896]:
!git push origin master

Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 8 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 9.00 KiB | 4.50 MiB/s, done.
Total 3 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/Luc-Bertin/TDs_ESILV.git
   6bd3f08..1eac305  master -> master


In [1254]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Discover Pandas 

Pandas is built on top of Numpy

First, check the overview of the package [here](https://pandas.pydata.org/docs/getting_started/overview.html)

>`pandas.DataFrame` is a 2-dimensional labeled data structure with columns of potentially different types. You can think of it like a spreadsheet or SQL table.
It got rows and columns' labels and can greatly contain and handle missing data.

Dealing with initially less structured, clean and complete data consists in most of the time spent by the data scientist.

In [1]:
import pandas as pd

In [2]:
pd.__version__

'0.25.3'

In [3]:
pd?

In [176]:
%%timeit
3+2

14.7 ns ± 0.386 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In the rest of this tutorial we will mainly work on the `DataFrame` class, although we first have to introduce 2 other core data structures provided by the package: the `Series` and the `Index`, as they are each constitutive of `DataFrame` and the former share very similar API with the DataFrame class.

## Series

one-dimensional array of indexed data. 

In [1007]:
pd.Series([3,2,1])

0    3
1    2
2    1
dtype: int64

with explicit index definition !

Example:

In [1008]:
serie_1 = pd.Series([3,2,1], index=[93,129, 219394])

In [1009]:
serie_1

93        3
129       2
219394    1
dtype: int64

In [1010]:
serie_1.index

Int64Index([93, 129, 219394], dtype='int64')

a dictionnary-like, object with possible keys repetition

In [1011]:
serie = pd.Series([3,2,1], index=["rené", "rené", "jean"])

In [1012]:
serie

rené    3
rené    2
jean    1
dtype: int64

In [1013]:
serie.values

array([3, 2, 1])

In [1014]:
serie.index

Index(['rené', 'rené', 'jean'], dtype='object')

* Access by key

In [1015]:
serie['rené']

rené    3
rené    2
dtype: int64

* Set a new key pair

In [1016]:
serie['joseph'] = 5

* Change a value for a key

In [1017]:
serie['rené'] = 4

In [1018]:
serie

rené      4
rené      4
jean      1
joseph    5
dtype: int64

In [1019]:
serie['rené'] = [4,3]

In [1020]:
serie

rené      4
rené      3
jean      1
joseph    5
dtype: int64

* delete a key val pair

In [1021]:
del serie["rené"]

In [1022]:
serie

jean      1
joseph    5
dtype: int64

In [1023]:
serie[0:4:2] # indexing: not possible in a simple dict 

jean    1
dtype: int64

* lookup 

In [1024]:
print('rené' in serie)
print("jean" in serie)

False
True


- When index is unique, pandas use a hashtable just like `dict`s : O(1). 
- When index is non-unique and sorted, pandas use binary search O(logN)
- When index is non-unique and not-sorted pandas need to check all the keys just like a list look-up: O(N).



using a `dict` in the `pd.Series` constructor automatically assigns the index as the ordered keys in the `dict` (for Python 3.6 and later though, the index is in the same order as the insertion order).


In [1025]:
test = pd.Series(dict(zip(["ea","fzf","aeif"], [2,3,2])))
# with zip or using a dict
test2 = pd.Series({"ea":2, "fzf":3, "aeif":2}, index=["ea"])

In [1026]:
test

aeif    2
ea      2
fzf     3
dtype: int64

In [1027]:
test2

ea    2
dtype: int64

**If multiple different types reside in a Series, all of the data will get upcasted to a dtype that accommodates all of the data involved.**



In [1030]:
test2 = pd.Series({"ea":2, "fzf":3, "aeif":"zf"}, index=["ea"])
test2

ea    2
dtype: object

In [1033]:
test2 = pd.Series({"ea":2, "fzf":3, "aeif":2.4}, index=["ea"])
test2

ea    2.0
dtype: float64

**dtype=object** means that the best common type infered representation for the contents of the `pd.Series` is that they are Python objects.

this also mean performance decreases, **any operations on the data will be done at the Python level** (1)

## selection in Series

In [1038]:
(test>2) # return a boolean array we can later use to filter the values we want (returning True)

aeif    False
ea      False
fzf      True
dtype: bool

In [1039]:
(test<4)

aeif    True
ea      True
fzf     True
dtype: bool

In [1041]:
# not "and" but "&" : & operator is a bitwise "and"
(test>2) & (test < 4) 

aeif    False
ea      False
fzf      True
dtype: bool

In [1042]:
type((test>2) & (test < 4) )

pandas.core.series.Series

In [1043]:
# mask ( the last expression whose result is an pd.Serie stored in the variable mask)
mask = (test>2) & (test < 4)

In [1044]:
test[mask]

fzf    3
dtype: int64

In [1046]:
# fancy indexing (<=> selecting multiple indexes using a list of indexes)
test[["ea", "fzf"]]

ea     2
fzf    3
dtype: int64

In [1048]:
# explicit index slicing (using the labels of the indexes)
test["aeif": "fzf"]

aeif    2
ea      2
fzf     3
dtype: int64

In [1049]:
# implicit index slicing (using integers)
test[0: 2]

aeif    2
ea      2
dtype: int64

- using explicit indexes while slicing ***include*** the final index

- using implicit index in slicing ***exclude*** the final index

What about i defined **explicit integer indexes at first** and i want to slice ? 🙄

## using loc

In [1061]:
serie2 = pd.Series({1:4, 2:8, 3:51})
serie2

1     4
2     8
3    51
dtype: int64

In [1076]:
serie2[3] # indexing: defaults to select explicit index /  with label 3
serie2[2:3] # slicing: defaults to select implicit index

51

3    51
dtype: int64

In [1085]:
serie2.loc[1] # indexing: on explicit index
serie2.loc[2:3] # slicing: on explicit index

4

2     8
3    51
dtype: int64

In [1086]:
serie2.iloc[1] # indexing: on implicit index
serie2.iloc[2:3] # slicing: on implicit index

8

3    51
dtype: int64

In [1090]:
serie2.loc[1:5] # slicing: explicit index 
serie2.iloc[1:5] # slicing: implicit index 

1     4
2     8
3    51
dtype: int64

2     8
3    51
dtype: int64

In [1094]:
serie2.loc[[1,2]] # fancy indexing
serie2.iloc[[1,2]] # fancy indexing 

1    4
2    8
dtype: int64

2     8
3    51
dtype: int64

### Index object 

* are immutable

In [1096]:
serie2.index[0]=18

TypeError: Index does not support mutable operations

* can be sliced or indexed (just like an array)

In [1104]:
serie2.index[0]

1

In [1105]:
serie2.index[:2]

Int64Index([1, 2], dtype='int64')

In [1106]:
serie2.index & {1, 5}

Int64Index([1], dtype='int64')

In [1107]:
serie2.index ^ {1,5}

Int64Index([2, 3, 5], dtype='int64')

# DataFrame

* sequence of "aligned" Series objects (sharing same indexes / like an Excel file )

* each Series object is a column

* Hence `pd.DataFrame` can be seen as dictionnary of Series objects

* Flexible rows and columns' labels (`Index` objects for both)

In [1205]:
serie1 = pd.Series({"Luc": 25, "Corentin":29, "René": 40})
serie2 = pd.Series({"René": "100%", "Corentin": "25%", "Luc": "20%"})

In [1206]:
# dictionnary of pd.Series
df = pd.DataFrame({"note": serie1, 
                   "charge_de_travail": serie2})

In [1207]:
df

,charge_de_travail,note
Corentin,25%,29
Luc,20%,25
René,100%,40


In [1208]:
# index objects on both columns and rows
df.index
df.columns

Index(['Corentin', 'Luc', 'René'], dtype='object')

Index(['charge_de_travail', 'note'], dtype='object')

> If you pass an index and / or columns, you are guaranteeing the index and / or columns of the resulting DataFrame. Thus, a dict of Series plus a specific index will discard all data not matching up to the passed index.

In [1209]:
df2 = pd.DataFrame({"note": serie1, 
                    "charge_de_travail": serie2}, 
                   index=["Corentin", "Luc", "Julie"],
                   columns=["note", "autre"])
df2 
# filled with NaN ("Not A Number") 
# when no value exist for the given (row_index, column_index)

,note,autre
Corentin,29.0,NaN
Luc,25.0,NaN
Julie,NaN,NaN


In [1210]:
df.shape

(3, 2)

shape: tuple of the number of elements with respect to each dimension

For a 1D array, the shape would be (n,) where n is the number of elements in your array.

For a 2D array, the shape would be (n,m) where n is the number of rows and m is the number of columns in your array

accessing a column/`Serie` by key : 

In [1211]:
df['note']

Corentin    29
Luc         25
René        40
Name: note, dtype: int64


Using the attribute notation is not advised for assignements as some methods or attributes of the same name already exist in the DataFrame class' own namespace

In [1212]:
df.note

Corentin    29
Luc         25
René        40
Name: note, dtype: int64

The `DataFrame` can be constructed using a list of dictionary
each dict element is a row
each key of each dict refers a column

In [1213]:
df2 = pd.DataFrame([{'a': 1, 'b': 2}, {'b': 3, 'c': 4}])
df2

,a,b,c
0,1.0,2,NaN
1,NaN,3,4.0


In [1214]:
pd.DataFrame([(1, 1, 3), (1, 2,4), (1,1,1)],
             columns=["a", "b", "c"],
            index=["Jean", "Jacques", "René"])

,a,b,c
Jean,1,1,3
Jacques,1,2,4
René,1,1,1


Indexing works the same way as for Series, but you have to account this time for the second dimension

`df.loc_or_iloc[ dim1 = rows, dim2 = columns]`


In [1215]:
df.iloc[:3, :1] # implicit indexing

,charge_de_travail
Corentin,25%
Luc,20%
René,100%


**columns slicing/indexing** is optional here, without specifying it, you **select only rows** 

In [1216]:
df.iloc[:3]

,charge_de_travail,note
Corentin,25%,29
Luc,20%,25
René,100%,40


In [1217]:
df.loc["Corentin":"Luc","charge_de_travail":"note"] # explicit indexing

,charge_de_travail,note
Corentin,25%,29
Luc,20%,25


same thing here, **only rows selected**

In [1218]:
df.loc[:"Corentin"]

,charge_de_travail,note
Corentin,25%,29


In [1219]:
df.loc[["Corentin", "Luc"], :] # mixing slicing and fancy indexing

,charge_de_travail,note
Corentin,25%,29
Luc,20%,25


Something to mention here, by default, without using indexers like `loc` and `iloc`:
- indexing directly `df`, performs the indexing on its columns **(ex:1)**
- slicing by conditions, or using a slice notation (::), is performed on rows **(ex:2)**

(1)

In [1220]:
df[["charge_de_travail"]] # indexing directly df defaults to columns

,charge_de_travail
Corentin,25%
Luc,20%
René,100%


(2) 

In [1221]:
mask = df["charge_de_travail"]=="25%" 
mask

Corentin     True
Luc         False
René        False
Name: charge_de_travail, dtype: bool

In [1222]:
df[mask] # masking directly df is operated on rows

,charge_de_travail,note
Corentin,25%,29


In [1223]:
df[:3] # slicing directly df is operated on rows

,charge_de_travail,note
Corentin,25%,29
Luc,20%,25
René,100%,40


## Operations on Pandas

Element-wise operations are made easy in `Pandas`.


* 3 - 2 <=> substract(3,2) <=> binary operation (2 inputs)
* -2 <=> neg(2) <=> unary operation (one input)
* sin(2) <=> unary operation (one input)

in `Pandas` : 
  - unary operations on `df`s elements preserve the indexes.
  - binary operations on elements from 2 `df`s align the operations on the indexes.

In [1224]:
import numpy as np 

In [1225]:
rng = np.random.RandomState(42) # for reproducibility
data = rng.randint(0,10, (3,4)) # creating an array of random integer values

In [1226]:
df = pd.DataFrame(data)
df

,0,1,2,3
0,6,3,7,4
1,6,9,2,6
2,7,4,3,7


In [1227]:
df2 = pd.DataFrame(rng.randint(0,10, (4,4)))
df2

,0,1,2,3
0,7,2,5,4
1,1,7,5,1
2,4,0,9,5
3,8,0,9,2


In [1228]:
df2 = df2.reindex([1,0,2,3]) #just to show rearranged indexes (does not change the association with the indexed data)
df2

,0,1,2,3
1,1,7,5,1
0,7,2,5,4
2,4,0,9,5
3,8,0,9,2


In [1229]:
df + df2

,0,1,2,3
0,13.0,5.0,12.0,8.0
1,7.0,16.0,7.0,7.0
2,11.0,4.0,12.0,12.0
3,NaN,NaN,NaN,NaN


on line of index 0, `7+6 = 13`
 which shows indexes had been aligned during the binary operation

also notice the union of the indices during the binary operation. If one may not exist in either of the dataframes and the result can't be evalutated, `NaN` fill the concerned entries

In [1230]:
df.__add__(df2, fill_value=25) # used in the binary operation 25+8 = 33)

,0,1,2,3
0,13.0,5.0,12.0,8.0
1,7.0,16.0,7.0,7.0
2,11.0,4.0,12.0,12.0
3,33.0,25.0,34.0,27.0


#### Operation between pandas series and a pandas dataframe

From the Numpy Docs

> Broadcasting is **how numpy treats arrays with different shapes during arithmetic operations**. 
Subject to certain constraints, the smaller array is “broadcast” across the larger array so that they **have compatible shapes.**
Broadcasting provides a means of vectorizing array operations so that looping occurs in C instead of Python

The only requirement for broadcasting is a way aligning array dimensions such that either :
* aligned dimensions are equal (so that operations are done on an element-by-element basis from 2 arrays of same shape)
* one of the aligned dimensions is 1 (in other words, dimensions with size 1 are stretched or “copied” to match the dimension of the other array)

Operations between `pandas.Series` and `pandas.DataFram` respect the numpy broadcasting rules:
>  If the two arrays **differ in their number of dimensions**, the shape of the one with **fewer dimensions is padded with ones on its leading (left) side.'**  (2)

In [1231]:
df.shape, df.iloc[1].shape, df.iloc[1][np.newaxis, :].shape

((3, 4), (4,), (1, 4))

In [1232]:
df
df.iloc[1]

,0,1,2,3
0,6,3,7,4
1,6,9,2,6
2,7,4,3,7


0    6
1    9
2    2
3    6
Name: 1, dtype: int64

In [1233]:
df - df.iloc[1] #row-wise (1,4) copied other 3 times => (3,4)

,0,1,2,3
0,0,-6,5,-2
1,0,0,0,0
2,1,-5,1,1


In [1234]:
df - df.iloc[1].sample(4) # again: kept the index alignements during computation

,0,1,2,3
0,0,-6,5,-2
1,0,0,0,0
2,1,-5,1,1


if you want to do it columnwise and not row wise

In [1236]:
df.__sub__(df.iloc[1], axis=0) # caution, the indexes operations will be based on the column indexes

,0,1,2,3
0,0.0,-3.0,1.0,-2.0
1,-3.0,0.0,-7.0,-3.0
2,5.0,2.0,1.0,5.0
3,NaN,NaN,NaN,NaN


In [1237]:
df.columns = ["a","b",0,"d"]
df

,a,b,0,d
0,6,3,7,4
1,6,9,2,6
2,7,4,3,7


In [1238]:
df.iloc[1]

a    6
b    9
0    2
d    6
Name: 1, dtype: int64

In [1240]:
df.__sub__(df.iloc[1], axis=0) 
# based on the column indexes
# only 0 match with one of the column index label

,a,b,0,d
0,4.0,1.0,5.0,2.0
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
a,NaN,NaN,NaN,NaN
b,NaN,NaN,NaN,NaN
d,NaN,NaN,NaN,NaN


In [1247]:
df[0].shape, df.shape

((3,), (3, 4))

In [1248]:
df2 = df - pd.DataFrame([(1,2), (4,5), (9,19)], columns=["a","b"])
df2

,0,a,b,d
0,NaN,5,1,NaN
1,NaN,2,4,NaN
2,NaN,-2,-15,NaN



### dtypes

In [862]:
print(df.dtypes)
print(df2.dtypes) 
# NaN is a floating-point value, 
# hence the Series embedding it gets its dtype upcasted to float (if it were an int)
# this pd.Series supports fast operations

a    int64
b    int64
0    int64
d    int64
dtype: object
0    float64
a      int64
b      int64
d    float64
dtype: object


In [863]:
%timeit np.arange(1E6, dtype="int").sum()

1.14 ms ± 142 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [864]:
%timeit np.arange(1E6, dtype="object").sum()

77.8 ms ± 4.15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [865]:
%timeit np.arange(1E6, dtype="float").sum()

1.25 ms ± 17.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Managing missing values

In [1250]:
pd.Series([2, np.nan]).isnull()

0    False
1     True
dtype: bool

In [1251]:
df2.iloc[0,2] = np.nan

In [1252]:
df2
df2.isnull()

,0,a,b,d
0,NaN,5,NaN,NaN
1,NaN,2,4.0,NaN
2,NaN,-2,-15.0,NaN


,0,a,b,d
0,True,False,True,True
1,True,False,False,True
2,True,False,False,True


In [1255]:
pd.Series([2, np.nan]).dropna()

0    2.0
dtype: float64

In [1257]:
df2
df2.dropna(axis=1) # drop a column when contains one NA value
df2.dropna(axis=0) # drop a row when contains one NA value
df2.dropna(axis=1, how="all") # drop a column when contains all NA value
df2.dropna(axis=1, thresh=3) # drop a column if below 3 non-NA value

,0,a,b,d
0,NaN,5,NaN,NaN
1,NaN,2,4.0,NaN
2,NaN,-2,-15.0,NaN


,a
0,5
1,2
2,-2


,0,a,b,d


,a,b
0,5,NaN
1,2,4.0
2,-2,-15.0


,a
0,5
1,2
2,-2


In [1259]:
df2
df2.fillna(value=2) #fill NA with specified value
# fill NA backwards 
# i.e. using the following non-null element
# to fill preceding NA ones
# defaults on rows basis
df2.fillna(method="bfill") 
df2.fillna(method="bfill", axis=1) # on column basis

,0,a,b,d
0,NaN,5,NaN,NaN
1,NaN,2,4.0,NaN
2,NaN,-2,-15.0,NaN


,0,a,b,d
0,2.0,5,2.0,2.0
1,2.0,2,4.0,2.0
2,2.0,-2,-15.0,2.0


,0,a,b,d
0,NaN,5,4.0,NaN
1,NaN,2,4.0,NaN
2,NaN,-2,-15.0,NaN


,0,a,b,d
0,5.0,5.0,NaN,NaN
1,2.0,2.0,4.0,NaN
2,-2.0,-2.0,-15.0,NaN


### MultiIndex

In [1260]:
data = {('group1', 'Luc'): 18,
        ('group2', 'Jean'): 23,
        ('group1', 'Seb'): 17,
        ('group1', 'René'): 4,
        ('group2', 'Alex'): 4,
        ('group3', 'Sophie'): 25,
        ('group2', 'Camille'): 2 }
serie = pd.Series(data)
serie

group1  Luc        18
        René        4
        Seb        17
group2  Alex        4
        Camille     2
        Jean       23
group3  Sophie     25
dtype: int64

In [1261]:
serie[:,"Luc"]

group1    18
dtype: int64

In [1271]:
serie["group1"]

Luc     18
René     4
Seb     17
dtype: int64

In [1273]:
serie[serie>=18]

group1  Luc       18
group2  Jean      23
group3  Sophie    25
dtype: int64

In [1266]:
# creating the multi-index using cartesian product
index = pd.MultiIndex.from_arrays([['group1', 'a', 'b', 'b'], ["Luc", 2, 1, 2]])

In [1268]:
serie.reindex(index) # works for multi-index too !
# Conform Series to new index with optional filling logic, placing
# NA/NaN in locations having no value in the previous index

group1  Luc    18.0
a       2       NaN
b       1       NaN
        2       NaN
dtype: float64

In [1269]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product(
    [['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],names=['subject', 'type'])
# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37
# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns) 
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      52.0  36.4  38.0  36.6  32.0  38.1
     2      28.0  37.7  47.0  35.4  50.0  36.4
2014 1      30.0  37.0  16.0  36.6  49.0  37.7
     2      52.0  36.8  31.0  35.5  36.0  37.6

In [1270]:
health_data.iloc[:1, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      52.0  36.4

In [981]:
#health_data.loc[(:,1),["Bob"]]

In [971]:
health_data.iloc[0,0]

39.0

### Merging